**Assignment 1: Basic text processing**

**Author- Hariharan Gopinath, Sanjeev madhavan, Colton Cunov**

**DAT450-Machine learning for natural language processing**

**Preprocessing**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NLP course/Assignment-1

/content/drive/MyDrive/NLP course/Assignment-1


In [ ]:
 #!unzip '/content/drive/MyDrive/NLP course/Assignment-1/a1_data.zip'

Archive:  /content/drive/MyDrive/NLP course/Assignment-1/a1_data.zip
replace a1_data/europarl.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import spacy
from collections import Counter, defaultdict
from matplotlib import pyplot as plt
import torch

### Warmup: computing word frequencies
#### most frequent words

In [ ]:
file_names = ['books.txt','europarl.txt','wikipedia.txt']
encodings = ['ISO-8859-1','utf-8','utf-8']
base_dir = 'C:/Users/Colton/OneDrive/School/Machine Learning for Natural Language Processing/HW_1/a1_data/' # fix

In [ ]:
def get_frequency_counter(file_name,encoding):
    freqs = Counter()
    with open(base_dir + file_name, encoding = encoding) as f:
        for line in f:
            tokens = line.lower().split()
            for token in tokens:
                freqs[token] += 1
    return freqs

In [ ]:
freqs = {}
for i in range(len(file_names)):
    file_name = file_names[i]
    encoding = encodings[i]
    freqs[file_name] = get_frequency_counter(file_name, encoding)
    print('--------------' + file_name + '-------------')
    for word, freq in freqs[file_name].most_common(10):
        print(word + '\t' + str(freq))

NameError: ignored

#### dictionary-within-dictionary

In [ ]:
print('Most common words following "red"')
for i in range(len(file_names)):
    file_name = file_names[i]
    encoding = encodings[i]
    freqs = defaultdict(Counter)
    with open(base_dir+file_name, encoding=encoding) as f:
        for line in f:
            tokens = line.lower().split()
            for t1, t2 in zip(tokens, tokens[1:]):
                freqs[t1][t2] += 1
    word = freqs['red'].most_common(1)[0][0]
    freq = freqs['red'].most_common(1)[0][1]
    print(file_name + ':\t' + word + '\t' + str(freq))

### Investigating the word frequency distribution
Our reflection goes here

In [ ]:
# make sure freqs is list of get_frequency_counter()
#  i.e. 
#   freqs = {}
#   for i in range(len(file_names)):
#     freqs[i] = get_frequency_counter(file_names[i], encodings[i])
word_freqs = []
plt.subplot(121)
for i in range(len(file_names)):
    file_name = file_names[i]
    word_freqs.append([x[1] for x in freqs[file_name].most_common(100)])
    plt.plot(word_freqs[i],label=file_name[:-4])
plt.legend()
plt.title('Linear')

plt.subplot(122)
for i in range(len(word_freqs)):
    plt.loglog(word_freqs[i],label=file_names[i][:-4])
plt.legend()
plt.title('Log-Log')
plt.show()

### Comparing corpora
*Is this asking us to find "words from corpora[i] that are common in corpora[j]"?*
*or maybe he wants a heatmap of frequency?*

In [ ]:
# freqs = {}
# for i in range(len(file_names)):
#     freqs[i] = get_frequency_counter(file_names[i], encodings[i])
for i in range(len(file_names)):
    file_name_1 = file_names[i]
    for j in range(len(file_names)):
        file_name_2 = file_names[j]
        if file_name_1 == file_name_2:
            continue
        print('\n')
        print('Top 10 words from "' + file_name_1 + '" (first) and their relative frequency in "' + file_name_2 + '" (second):')
        print('WORD\tFREQ\tFREQ')
        print('--------------------')
        top_10_counter = freqs[file_name_1].most_common(10)
        top_10_words = [x[0] for x in top_10_counter]
        relative_freq_1 = [x[1] / sum(freqs[file_name_1].values()) for x in top_10_counter]
        relative_freq_2 = []
        for word in top_10_words:
            relative_freq_2.append(freqs[file_name_2][word] / sum(freqs[file_name_2].values()))
        for i in range(len(top_10_words)):
            print(top_10_words[i] + '\t' + str(round(100*relative_freq_1[i],2)) + '%\t' + str(round(100*relative_freq_2[i],2)) + '%')
                    

### Side show: preprocessing text for machine learning

In [ ]:
class Vocab:
    def __init__(self, dataset, max_voc_size=1000, batch_size=1000):
        ############# word-to-int mapping #############
        self.dataset = dataset
        self.max_voc_size = max_voc_size
        freqs = Counter()
        tokens = dataset.lower().split()
        for token in tokens:
            freqs[token] += 1
        # freqs is dict(), i.e. freqs[word] = count
        if len(freqs) > max_voc_size:
            voc_size = max_voc_size
        else:
            voc_size = len(freqs)
        freqs = freqs.most_common(voc_size)
        # now freqs is tuple of (word, count)
        mapping = {freqs[i][0]:i+1 for i in range(voc_size)}
        # now freqs is dict of freqs[word]=r where r=rank based on frequency, r=(1,...)
        self.mapping = mapping # should use self.mapping.get('key',-1) to try non-existent keys

        
        ############# batching #####################
        split_dataset = dataset.split('\n')
        if len(split_dataset) > batch_size:
            split_dataset = split_dataset[:batch_size]
        else:
            batch_size = len(split_dataset)

        max_line_length = len(max(split_dataset, key=len))
        batches = torch.ones(max_line_length, batch_size) * -1 # -1 is the padding
        for line_idx in range(len(split_dataset)):
            line = split_dataset[line_idx]
            split_line = line.split()
            for word_idx in range(len(split_line)):
                batches[word_idx][line_idx] = mapping.get(split_line[word_idx], 0) # 0 is what words not in the mapping map to
        self.batches = batches
            
        
i = 0
max_voc_size=1000
batch_size=1000
with open(base_dir+file_names[i], encoding=encodings[i]) as f:
    dataset = f.read()
    
vc = Vocab(dataset, max_voc_size)
print(vc.mapping)
print(vc.batches)

### Trying out an NLP toolkit

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
# !python -m spacy download en

#### Processing a text

In [ ]:
example = 'Apple bought two companies this year and no one knew, Mark Gurman at 9to5Mac reports.'
result = nlp(example)

In [ ]:
from spacy import displacy
displacy.render(result, style='ent', jupyter=True)
# displacy.render(result, style='dep', jupyter=True)

In [ ]:
for token in result:
    print('Text: ' + token.text + 
          '\tPoS: ' + token.pos_ + 
          '\tLemma: ' + token.lemma_ + 
          '\tHead: ' + str(token.head) + 
          '\tDep: ' + str(token.dep_))
    
print('----------')
for entity in result.ents:
    print(entity)

Obviously, "two" is not a named entity. It got it right that "one" in this case is a noun, not a number while "two" is a number. However "9to5Mac" is not a number. Also, "reports" is a verb not a noun.

### Additional questions

In [ ]:
corpora = {}
for i in range(len(file_names)):
    with open(base_dir+file_names[i],encoding=encodings[i]) as f:
        corpora[file_names[i]] = f.read()

#### Which are the most frequent nouns in the book review corpus?
SpaCy tags entities as GPE (GeoPolitical Entity?) if they are countries, states, cities, provinces, etc. We could use geonamescache to validate if a token is actually a country, but I don't think that's what they really want us to do

In [ ]:
num_lines = 1000
nouns = Counter()
for line in corpora['books.txt'].split('\n')[:num_lines]:
    result = nlp(line)
    for token in result:
        if token.pos_ == 'NOUN':
            nouns[token.text] += 1
print(nouns.most_common(10))

#### Which are the most frequently mentioned countries in the Wikipedia corpus?
SpaCy tags entities as GPE (GeoPolitical Entity?) if they are countries, states, cities, provinces, etc.
We could use *geonamescache* to validate if a token is actually a country, but I don't think that's what they really want us to do

In [ ]:
num_lines = 1000
countries = Counter()
for line in corpora['wikipedia.txt'].split('\n')[:num_lines]:
    result = nlp(line)
    for entity in result.ents:
        if entity.label_ == 'GPE':
            countries[entity.text] += 1
print(countries.most_common(10))

#### What are the items that people drink most frequently in the European Parliament corpus?
To count this, we look at words with POS=noun, HEAD = inflection of *drink*, and DEPENDENCY = "dobj"

In [ ]:
inflected_forms = ['drink','drank','drunk','drinking','drinks','Drink','Drank','Drunk','Drinking','Drinks']
drinks = Counter()
for line in corpora['europarl.txt'].split('\n'):
    for inflected_form in inflected_forms:
        if inflected_form in line:
            result = nlp(line)
            for token in result:
                if token.pos_=='NOUN' and \
                    str(token.head) == inflected_form and \
                    token.dep_ == 'dobj' and \
                    token.text != inflected_form: #idk if this last one is necessary
                    drinks[token.text] += 1
print(drinks.most_common(10))